In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import urllib
import requests
# from PyPDF2 import PdfMerger
from urllib.parse import urlparse, urlunparse
from io import BytesIO
from enum import Enum
import re
from urllib.error import URLError, HTTPError  # Import these classes from urllib.error
import ssl
import pymupdf
import os
import undetected_chromedriver as uc


from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import tempfile
import time
import xml.etree.ElementTree as ET
import grobid_tei_xml
import json

## Extract Link from 150 Good research paper

In [2]:
# Read in the csv file
## This is a csv file that teammates manually inspected as good research pape
good_df = pd.read_csv("../../data/150_research_papers.csv")[['Link']]
#Add the label -> 1 for relevent
good_df['label'] = 1

In [3]:
good_df.head()
# good_df.shape

,Link,label
0,https://www.science.org/doi/10.1126/science.ad...,1
1,https://www.nature.com/articles/s41566-019-0398-2,1
2,https://www.nature.com/articles/s41560-020-007...,1
3,https://www.science.org/doi/10.1126/science.ab...,1
4,https://www.nature.com/articles/s41467-021-236...,1


## Import bad paper

In [4]:
bad_df = pd.read_csv("../../data/irrelevant_papers.csv")
bad_df.head()

,link
0,https://doi.org/10.1016/0021-9797(77)90150-3
1,https://www.sciencedirect.com/science/article/...
2,https://www.sciencedirect.com/science/article/...
3,https://doi.org/10.1107%2FS0021889813019535
4,https://www.sciencedirect.com/science/article/...


In [5]:
#Add the label -> 0 for irrelevent
bad_df['label'] = 0
# sampled_bad_df.to_csv('bad_paper_small.csv', index=False)

In [6]:
bad_df

,link,label
0,https://doi.org/10.1016/0021-9797(77)90150-3,0
1,https://www.sciencedirect.com/science/article/...,0
2,https://www.sciencedirect.com/science/article/...,0
3,https://doi.org/10.1107%2FS0021889813019535,0
4,https://www.sciencedirect.com/science/article/...,0
...,...,...
310,https://www.sciencedirect.com/science/article/...,0
311,https://www.sciencedirect.com/science/article/...,0
312,https://www.sciencedirect.com/science/article/...,0
313,https://doi.org/10.1063/1.1404988,0


# Scraping Text

In [7]:
download_dir = tempfile.mkdtemp()
chrome_options = uc.ChromeOptions()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,  # Set download location
    "download.prompt_for_download": False,       # Disable download prompts
    "plugins.always_open_pdf_externally": True   # Download PDFs instead of opening them
})
driver = uc.Chrome(options=chrome_options)

In [8]:
def get_text(soup):
    #Given a beautiful soup object, it will extact the text
    for script in soup(['script', 'style']):
        script.extract()
    text = soup.get_text(separator=' ')
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

In [9]:
def extract_text_from_urls(url, paper_index):

    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    text_extracted = get_text(soup)
    return text_extracted

In [10]:
#Record: took 7min 40sec
text_good_list = []
for index, row in good_df.iterrows():
    text_gotten = extract_text_from_urls(row['Link'], index)
    print(text_gotten[:30])
    text_good_list.append(text_gotten)

Improved charge extraction in 
Surface passivation of perovsk
Intact 2D/3D halide junction p
Deterministic fabrication of 3
Multication perovskite 2D/3D i
Front-contact passivation thro
Ligand-Stabilized Reduced-Dime
The 2D Halide Perovskite Ruleb
Reconfiguring the band-edge st
Defect Passivation of Organic–
Stability-limiting heterointer
3D Perovskite Passivation with
Record Open‐Circuit Voltage Wi
High‐Performance Flexible All‐
23.7% Efficient inverted perov
Imaging and quantifying non-ra
Deep surface passivation for e
Perovskite Quantum Wells Forma
Revealing the Transient Format
Engineering ligand reactivity 
How Stable Is the 2D/3D Interf
An interface stabilized perovs
Interfacial engineering from m
Quantum-size-tuned heterostruc
Accelerated aging of all-inorg
Thermal degradation of formami
On the Thermal and Thermodynam
Inorganic wide-bandgap perovsk
Thermal unequilibrium of strai
Amine additive reactions induc
Ammonia for post-healing of fo
Surface reaction for efficient
Unique P

TimeoutException: Message: timeout: Timed out receiving message from renderer: 298.686
  (Session info: chrome=131.0.6778.70)
Stacktrace:
	GetHandleVerifier [0x00655093+25075]
	(No symbol) [0x005DE124]
	(No symbol) [0x004BBE63]
	(No symbol) [0x004AD723]
	(No symbol) [0x004AD431]
	(No symbol) [0x004AB8CB]
	(No symbol) [0x004ABF3D]
	(No symbol) [0x004B81DA]
	(No symbol) [0x004C78A5]
	(No symbol) [0x004CC706]
	(No symbol) [0x004AC545]
	(No symbol) [0x004C7762]
	(No symbol) [0x0053B7C8]
	(No symbol) [0x00521C16]
	(No symbol) [0x004F3F3C]
	(No symbol) [0x004F4ECD]
	GetHandleVerifier [0x00942523+3094147]
	GetHandleVerifier [0x00955754+3172532]
	GetHandleVerifier [0x0094DF32+3141778]
	GetHandleVerifier [0x006F2100+668256]
	(No symbol) [0x005E6C4D]
	(No symbol) [0x005E3DF8]
	(No symbol) [0x005E3F95]
	(No symbol) [0x005D6C80]
	BaseThreadInitThunk [0x75837BA9+25]
	RtlInitializeExceptionChain [0x7716C0CB+107]
	RtlClearBits [0x7716C04F+191]


In [ ]:
##EXTRACTED ALL TEXT!!!
len(text_good_list)

151

In [ ]:
#Took 14 sec
text_bad_list = []
for index, row in bad_df.iterrows():
    text_gotten = extract_text_from_urls(row['link'], index)
    print(len(text_gotten))
    text_bad_list.append(text_gotten)

3456
3360
124571
38426
3360
27620
240378
3487
165537
9699
3357
60721
7914
3360
181061
3357
2909
84675
3360
122218
124594
37058
3360
42332
35463
2759
221397
48858
64718
3360
122218
124594
124594
2756
3357
2130
63994
9455
26402
3357
119653
42061
124404
4581
87808
8055
3357
75586
176638
3357
9930
3357
1008041
3360
122176
40178
237419
3357
122218
4070
114775
3357
252200
3357
122176
461560
59549
1069961
3360
2084
1869
52088
3357
122197
2976
157582
3360
122197
123831
124571
2421
3357
79645
3357
122176
161871
3357
122176
123830
149310
3357
122176
9990
197065
3360
50683
3360
122176
124641
124594
105218
63381
47550
72723
3357
124593
50217
160599
3357
50217
58486
61619
8716
3360
124571
123831
2716
37000
6780
3357
322502
24124
9422
26662
61184
79769
3360
3287
138795
3360
53943
3357
8981
3357
122176
123831
8431
2889
72284
65101
3360
8632
6698
3357
153786
162065
3357
122218
9810
3360
122218
124593
75650
3360
40632
15799
48458
12371
3357
113
114688
38357
8430
73090
175357
3360
120289
3360
124593
123

In [ ]:
len(text_bad_list)

315

In [ ]:
good_df['text'] = text_good_list
bad_df['text'] = text_bad_list

In [ ]:
good_df

NameError: name 'good_df' is not defined

In [ ]:
bad_df

In [ ]:
bad_df.to_csv('../../data/text_bad_paper.csv', index=False)

In [ ]:
good_df.to_csv('../../data/text_good_paper.csv', index=False)

In [ ]:
merged_df = good_df.append(bad_df, ignore_index=True)
merged_df

In [ ]:
merged_df.to_csv('../../merged_label.csv', index=False)